In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Masking,BatchNormalization, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert import BertTokenizer
from keras.callbacks import EarlyStopping
import codecs
from keras.layers.convolutional import Conv2D
import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)


# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [3]:
# other parameters
#MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 768 #1024
nb_words = 3438
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# word_index = tokenizer.word_index
# nb_words = len(word_index) +1
# print('Found %s unique tokens' % len(word_index))

def get_bert_embed_matrix():
    bert = BertModel.from_pretrained(BERT_FP)
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


def get_bert_embed_matrix():
    bert = BertModel.from_pretrained('bert-base-uncased')
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


g_word_embedding_matrix = get_bert_embed_matrix()

# # load model
# EMBEDDING_DIM = 300
# file_loc = '../../data/glove.42B.300d.txt'
# print (file_loc)

# gembeddings_index = {}
# with codecs.open(file_loc, encoding='utf-8') as f:
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         gembedding = np.asarray(values[1:], dtype='float32')
#         gembeddings_index[word] = gembedding
# #
# f.close()
# print('G Word embeddings:', len(gembeddings_index))

# nb_words = len(word_index) +1
# g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     gembedding_vector = gembeddings_index.get(word)
#     if gembedding_vector is not None:
#         g_word_embedding_matrix[i] = gembedding_vector
        
# print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
# split = 7869
# earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
#                           restore_best_weights=True)

# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_text = x_train_text[mask]
# model: GRU


In [4]:
#split train/test
split = 7869

In [5]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [6]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(*g_word_embedding_matrix.shape,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = Conv2D(64, 3, strides=(2, 2), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(2, 2), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(2, 2), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(2, 2), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(2, 2), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_speech, model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [7]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split],x_train_mocap[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('stmBERT_multi7869-3.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 209s 33ms/step - loss: 2.9952 - v_loss: 0.9995 - a_loss: 0.9970 - d_loss: 0.9988 - v_ccc: 5.3176e-04 - a_ccc: 0.0031 - d_ccc: 0.0012 - val_loss: 2.9998 - val_v_loss: 1.0000 - val_a_loss: 1.0000 - val_d_loss: 0.9999 - val_v_ccc: 5.7026e-05 - val_a_ccc: 5.0695e-05 - val_d_ccc: 5.7489e-05
Epoch 2/50
6295/6295 [==============================] - 212s 34ms/step - loss: 1.9098 - v_loss: 0.7153 - a_loss: 0.5618 - d_loss: 0.6313 - v_ccc: 0.2843 - a_ccc: 0.4377 - d_ccc: 0.3682 - val_loss: 1.7354 - val_v_loss: 0.7045 - val_a_loss: 0.4813 - val_d_loss: 0.5595 - val_v_ccc: 0.3020 - val_a_ccc: 0.5213 - val_d_ccc: 0.4413
Epoch 3/50
6295/6295 [==============================] - 216s 34ms/step - loss: 1.1656 - v_loss: 0.3966 - a_loss: 0.3376 - d_loss: 0.4309 - v_ccc: 0.6033 - a_ccc: 0.6623 - d_ccc: 0.5688 - val_loss: 1.6

6295/6295 [==============================] - 196s 31ms/step - loss: 1.3984 - v_loss: 0.5461 - a_loss: 0.3796 - d_loss: 0.4727 - v_ccc: 0.4537 - a_ccc: 0.6204 - d_ccc: 0.5276 - val_loss: 1.7020 - val_v_loss: 0.6742 - val_a_loss: 0.4999 - val_d_loss: 0.5385 - val_v_ccc: 0.3320 - val_a_ccc: 0.5036 - val_d_ccc: 0.4624
Epoch 3/50
6295/6295 [==============================] - 224s 36ms/step - loss: 1.1262 - v_loss: 0.3767 - a_loss: 0.3280 - d_loss: 0.4214 - v_ccc: 0.6234 - a_ccc: 0.6718 - d_ccc: 0.5787 - val_loss: 1.7627 - val_v_loss: 0.7955 - val_a_loss: 0.4686 - val_d_loss: 0.5053 - val_v_ccc: 0.2068 - val_a_ccc: 0.5334 - val_d_ccc: 0.4971
Epoch 4/50
6295/6295 [==============================] - 229s 36ms/step - loss: 1.0292 - v_loss: 0.3153 - a_loss: 0.3143 - d_loss: 0.3993 - v_ccc: 0.6847 - a_ccc: 0.6858 - d_ccc: 0.6004 - val_loss: 1.5487 - val_v_loss: 0.5877 - val_a_loss: 0.4369 - val_d_loss: 0.5322 - val_v_ccc: 0.4123 - val_a_ccc: 0.5673 - val_d_ccc: 0.4717
Epoch 5/50
6295/6295 [========

6295/6295 [==============================] - 232s 37ms/step - loss: 0.6343 - v_loss: 0.1635 - a_loss: 0.2344 - d_loss: 0.2374 - v_ccc: 0.8366 - a_ccc: 0.7662 - d_ccc: 0.7629 - val_loss: 1.6000 - val_v_loss: 0.6018 - val_a_loss: 0.4369 - val_d_loss: 0.5767 - val_v_ccc: 0.4030 - val_a_ccc: 0.5671 - val_d_ccc: 0.4299
Epoch 12/50
6295/6295 [==============================] - 233s 37ms/step - loss: 0.5952 - v_loss: 0.1540 - a_loss: 0.2214 - d_loss: 0.2201 - v_ccc: 0.8460 - a_ccc: 0.7789 - d_ccc: 0.7799 - val_loss: 1.5660 - val_v_loss: 0.5499 - val_a_loss: 0.4396 - val_d_loss: 0.5911 - val_v_ccc: 0.4561 - val_a_ccc: 0.5629 - val_d_ccc: 0.4150
Epoch 13/50
6295/6295 [==============================] - 232s 37ms/step - loss: 0.5690 - v_loss: 0.1436 - a_loss: 0.2099 - d_loss: 0.2154 - v_ccc: 0.8564 - a_ccc: 0.7900 - d_ccc: 0.7846 - val_loss: 1.5775 - val_v_loss: 0.5319 - val_a_loss: 0.4535 - val_d_loss: 0.6041 - val_v_ccc: 0.4681 - val_a_ccc: 0.5519 - val_d_ccc: 0.4025
Epoch 14/50
6295/6295 [=====

Epoch 1/50
6295/6295 [==============================] - 249s 39ms/step - loss: 2.9957 - v_loss: 0.9981 - a_loss: 0.9996 - d_loss: 0.9979 - v_ccc: 0.0020 - a_ccc: 2.8497e-04 - d_ccc: 0.0021 - val_loss: 2.8921 - val_v_loss: 1.0267 - val_a_loss: 0.9905 - val_d_loss: 0.8719 - val_v_ccc: -0.0323 - val_a_ccc: 0.0100 - val_d_ccc: 0.1303
Epoch 2/50
6295/6295 [==============================] - 241s 38ms/step - loss: 1.7490 - v_loss: 0.6835 - a_loss: 0.4851 - d_loss: 0.5798 - v_ccc: 0.3161 - a_ccc: 0.5148 - d_ccc: 0.4202 - val_loss: 1.8603 - val_v_loss: 0.7732 - val_a_loss: 0.5011 - val_d_loss: 0.5994 - val_v_ccc: 0.2307 - val_a_ccc: 0.5048 - val_d_ccc: 0.4042
Epoch 3/50
6295/6295 [==============================] - 241s 38ms/step - loss: 1.1251 - v_loss: 0.3651 - a_loss: 0.3371 - d_loss: 0.4231 - v_ccc: 0.6349 - a_ccc: 0.6628 - d_ccc: 0.5772 - val_loss: 1.6165 - val_v_loss: 0.6361 - val_a_loss: 0.4302 - val_d_loss: 0.5641 - val_v_ccc: 0.3641 - val_a_ccc: 0.5784 - val_d_ccc: 0.4410
Epoch 4/50
629

6295/6295 [==============================] - 211s 34ms/step - loss: 0.6520 - v_loss: 0.1681 - a_loss: 0.2387 - d_loss: 0.2451 - v_ccc: 0.8318 - a_ccc: 0.7611 - d_ccc: 0.7550 - val_loss: 1.5754 - val_v_loss: 0.5737 - val_a_loss: 0.4422 - val_d_loss: 0.5699 - val_v_ccc: 0.4286 - val_a_ccc: 0.5596 - val_d_ccc: 0.4365
Epoch 12/50
6295/6295 [==============================] - 214s 34ms/step - loss: 0.6058 - v_loss: 0.1582 - a_loss: 0.2213 - d_loss: 0.2261 - v_ccc: 0.8418 - a_ccc: 0.7787 - d_ccc: 0.7738 - val_loss: 1.5591 - val_v_loss: 0.5771 - val_a_loss: 0.4460 - val_d_loss: 0.5502 - val_v_ccc: 0.4268 - val_a_ccc: 0.5575 - val_d_ccc: 0.4567
Epoch 13/50
6295/6295 [==============================] - 214s 34ms/step - loss: 0.5737 - v_loss: 0.1498 - a_loss: 0.2065 - d_loss: 0.2179 - v_ccc: 0.8503 - a_ccc: 0.7938 - d_ccc: 0.7823 - val_loss: 1.6287 - val_v_loss: 0.5966 - val_a_loss: 0.4539 - val_d_loss: 0.5915 - val_v_ccc: 0.4067 - val_a_ccc: 0.5512 - val_d_ccc: 0.4134
Epoch 14/50
6295/6295 [=====

...

2170/2170 [==============================] - 10s 5ms/step
[0.4127190411090851, 0.5240565538406372, 0.4111548066139221]
24
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 236s 38ms/step - loss: 2.9183 - v_loss: 0.9856 - a_loss: 0.9645 - d_loss: 0.9678 - v_ccc: 0.0144 - a_ccc: 0.0355 - d_ccc: 0.0318 - val_loss: 2.1884 - val_v_loss: 0.8915 - val_a_loss: 0.6240 - val_d_loss: 0.6878 - val_v_ccc: 0.1090 - val_a_ccc: 0.3845 - val_d_ccc: 0.3181
Epoch 2/50
6295/6295 [==============================] - 221s 35ms/step - loss: 1.4648 - v_loss: 0.5675 - a_loss: 0.4017 - d_loss: 0.4945 - v_ccc: 0.4320 - a_ccc: 0.5980 - d_ccc: 0.5051 - val_loss: 1.7164 - val_v_loss: 0.6809 - val_a_loss: 0.4927 - val_d_loss: 0.5536 - val_v_ccc: 0.3262 - val_a_ccc: 0.5100 - val_d_ccc: 0.4474
Epoch 3/50
6295/6295 [==============================] - 221s 35ms/step - loss: 1.1285 - v_loss: 0.3724 - a_loss: 0.3307 - d_loss: 0.4252 - v_ccc: 0.6273 - a_ccc: 0.6693 - d_ccc

Epoch 10/50
6295/6295 [==============================] - 229s 36ms/step - loss: 0.7093 - v_loss: 0.1802 - a_loss: 0.2613 - d_loss: 0.2681 - v_ccc: 0.8199 - a_ccc: 0.7389 - d_ccc: 0.7319 - val_loss: 1.5818 - val_v_loss: 0.5896 - val_a_loss: 0.4392 - val_d_loss: 0.5706 - val_v_ccc: 0.4159 - val_a_ccc: 0.5654 - val_d_ccc: 0.4369
Epoch 11/50
6295/6295 [==============================] - 224s 36ms/step - loss: 0.6711 - v_loss: 0.1717 - a_loss: 0.2467 - d_loss: 0.2523 - v_ccc: 0.8282 - a_ccc: 0.7532 - d_ccc: 0.7476 - val_loss: 1.5439 - val_v_loss: 0.5979 - val_a_loss: 0.4277 - val_d_loss: 0.5330 - val_v_ccc: 0.4068 - val_a_ccc: 0.5737 - val_d_ccc: 0.4756
Epoch 12/50
6295/6295 [==============================] - 221s 35ms/step - loss: 0.6321 - v_loss: 0.1620 - a_loss: 0.2325 - d_loss: 0.2380 - v_ccc: 0.8381 - a_ccc: 0.7678 - d_ccc: 0.7619 - val_loss: 1.6035 - val_v_loss: 0.5914 - val_a_loss: 0.4507 - val_d_loss: 0.5776 - val_v_ccc: 0.4162 - val_a_ccc: 0.5499 - val_d_ccc: 0.4304
Epoch 13/50
6295

6295/6295 [==============================] - 207s 33ms/step - loss: 0.5336 - v_loss: 0.1406 - a_loss: 0.1927 - d_loss: 0.2008 - v_ccc: 0.8594 - a_ccc: 0.8076 - d_ccc: 0.7993 - val_loss: 1.5829 - val_v_loss: 0.5417 - val_a_loss: 0.4716 - val_d_loss: 0.5744 - val_v_ccc: 0.4558 - val_a_ccc: 0.5313 - val_d_ccc: 0.4301
Epoch 16/50
6295/6295 [==============================] - 208s 33ms/step - loss: 0.5076 - v_loss: 0.1335 - a_loss: 0.1835 - d_loss: 0.1905 - v_ccc: 0.8665 - a_ccc: 0.8164 - d_ccc: 0.8095 - val_loss: 1.6396 - val_v_loss: 0.5572 - val_a_loss: 0.5006 - val_d_loss: 0.5924 - val_v_ccc: 0.4449 - val_a_ccc: 0.5025 - val_d_ccc: 0.4131
Epoch 17/50
6295/6295 [==============================] - 207s 33ms/step - loss: 0.4789 - v_loss: 0.1279 - a_loss: 0.1717 - d_loss: 0.1794 - v_ccc: 0.8721 - a_ccc: 0.8283 - d_ccc: 0.8207 - val_loss: 1.6114 - val_v_loss: 0.5422 - val_a_loss: 0.4849 - val_d_loss: 0.5968 - val_v_ccc: 0.4596 - val_a_ccc: 0.5190 - val_d_ccc: 0.4100
Epoch 18/50
6295/6295 [=====

6295/6295 [==============================] - 207s 33ms/step - loss: 0.5223 - v_loss: 0.1330 - a_loss: 0.1993 - d_loss: 0.1902 - v_ccc: 0.8670 - a_ccc: 0.8008 - d_ccc: 0.8099 - val_loss: 1.5192 - val_v_loss: 0.5184 - val_a_loss: 0.4466 - val_d_loss: 0.5628 - val_v_ccc: 0.4764 - val_a_ccc: 0.5606 - val_d_ccc: 0.4438
Epoch 17/50
6295/6295 [==============================] - 204s 32ms/step - loss: 0.4901 - v_loss: 0.1257 - a_loss: 0.1858 - d_loss: 0.1790 - v_ccc: 0.8744 - a_ccc: 0.8144 - d_ccc: 0.8211 - val_loss: 1.5773 - val_v_loss: 0.5436 - val_a_loss: 0.4567 - val_d_loss: 0.5893 - val_v_ccc: 0.4522 - val_a_ccc: 0.5531 - val_d_ccc: 0.4174
Epoch 18/50
6295/6295 [==============================] - 212s 34ms/step - loss: 0.4655 - v_loss: 0.1201 - a_loss: 0.1750 - d_loss: 0.1705 - v_ccc: 0.8800 - a_ccc: 0.8249 - d_ccc: 0.8295 - val_loss: 1.5647 - val_v_loss: 0.5410 - val_a_loss: 0.4460 - val_d_loss: 0.5925 - val_v_ccc: 0.4582 - val_a_ccc: 0.5615 - val_d_ccc: 0.4157
Epoch 19/50
6295/6295 [=====

6295/6295 [==============================] - 202s 32ms/step - loss: 0.5526 - v_loss: 0.1428 - a_loss: 0.2014 - d_loss: 0.2089 - v_ccc: 0.8572 - a_ccc: 0.7988 - d_ccc: 0.7913 - val_loss: 1.5621 - val_v_loss: 0.5440 - val_a_loss: 0.4508 - val_d_loss: 0.5787 - val_v_ccc: 0.4587 - val_a_ccc: 0.5531 - val_d_ccc: 0.4261
Epoch 15/50
6295/6295 [==============================] - 206s 33ms/step - loss: 0.5250 - v_loss: 0.1371 - a_loss: 0.1887 - d_loss: 0.1991 - v_ccc: 0.8629 - a_ccc: 0.8112 - d_ccc: 0.8009 - val_loss: 1.5720 - val_v_loss: 0.5539 - val_a_loss: 0.4556 - val_d_loss: 0.5730 - val_v_ccc: 0.4481 - val_a_ccc: 0.5477 - val_d_ccc: 0.4322
Epoch 16/50
6295/6295 [==============================] - 207s 33ms/step - loss: 0.5025 - v_loss: 0.1315 - a_loss: 0.1797 - d_loss: 0.1911 - v_ccc: 0.8685 - a_ccc: 0.8202 - d_ccc: 0.8088 - val_loss: 1.5968 - val_v_loss: 0.5357 - val_a_loss: 0.4678 - val_d_loss: 0.5951 - val_v_ccc: 0.4639 - val_a_ccc: 0.5353 - val_d_ccc: 0.4040
Epoch 17/50
6295/6295 [=====